In [1]:
import pandas as pd

# Read the CSV and Perform Basic Data Cleaning

In [2]:
df = pd.read_csv("cumulative.csv")
df = df.drop(columns=["rowid", "kepid", "kepoi_name", "kepler_name", "koi_pdisposition", "koi_score", "koi_tce_delivname"])
# Drop the null columns where all values are null
df = df.dropna(axis='columns', how='all')
# Drop the null rows
df = df.dropna()
df.head()

,koi_disposition,koi_fpflag_nt,koi_fpflag_ss,koi_fpflag_co,koi_fpflag_ec,koi_period,koi_period_err1,koi_period_err2,koi_time0bk,koi_time0bk_err1,...,koi_steff_err2,koi_slogg,koi_slogg_err1,koi_slogg_err2,koi_srad,koi_srad_err1,koi_srad_err2,ra,dec,koi_kepmag
0,CONFIRMED,0,0,0,0,9.488036,2.775000e-05,-2.775000e-05,170.538750,0.002160,...,-81.0,4.467,0.064,-0.096,0.927,0.105,-0.061,291.93423,48.141651,15.347
1,CONFIRMED,0,0,0,0,54.418383,2.479000e-04,-2.479000e-04,162.513840,0.003520,...,-81.0,4.467,0.064,-0.096,0.927,0.105,-0.061,291.93423,48.141651,15.347
2,FALSE POSITIVE,0,1,0,0,19.899140,1.494000e-05,-1.494000e-05,175.850252,0.000581,...,-176.0,4.544,0.044,-0.176,0.868,0.233,-0.078,297.00482,48.134129,15.436
3,FALSE POSITIVE,0,1,0,0,1.736952,2.630000e-07,-2.630000e-07,170.307565,0.000115,...,-174.0,4.564,0.053,-0.168,0.791,0.201,-0.067,285.53461,48.285210,15.597
4,CONFIRMED,0,0,0,0,2.525592,3.761000e-06,-3.761000e-06,171.595550,0.001130,...,-211.0,4.438,0.070,-0.210,1.046,0.334,-0.133,288.75488,48.226200,15.509


In [3]:
# Remove Space for `FALSE POSITIVE` category
mask = df["koi_disposition"] == "FALSE POSITIVE"
df.loc[mask, "koi_disposition"] = "False_Positive"
df["koi_disposition"]

0            CONFIRMED
1            CONFIRMED
2       False_Positive
3       False_Positive
4            CONFIRMED
5            CONFIRMED
6            CONFIRMED
7            CONFIRMED
8       False_Positive
9            CONFIRMED
10           CONFIRMED
11           CONFIRMED
12           CONFIRMED
13           CONFIRMED
14      False_Positive
15      False_Positive
16      False_Positive
17      False_Positive
18           CONFIRMED
19           CONFIRMED
20      False_Positive
21           CONFIRMED
22           CONFIRMED
23           CONFIRMED
24      False_Positive
25           CONFIRMED
26           CONFIRMED
27           CONFIRMED
28      False_Positive
29      False_Positive
             ...      
9532    False_Positive
9533         CANDIDATE
9535    False_Positive
9536    False_Positive
9537    False_Positive
9538    False_Positive
9539         CANDIDATE
9540         CONFIRMED
9541    False_Positive
9542         CANDIDATE
9543    False_Positive
9544         CANDIDATE
9545    Fal

# Create a Train Test Split

Use `koi_disposition` for the y values

In [4]:
from sklearn.model_selection import train_test_split
y = df["koi_disposition"]
X = df.drop(columns=["koi_disposition"])

X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=1, stratify=y)

In [5]:
X_train.head()

,koi_fpflag_nt,koi_fpflag_ss,koi_fpflag_co,koi_fpflag_ec,koi_period,koi_period_err1,koi_period_err2,koi_time0bk,koi_time0bk_err1,koi_time0bk_err2,...,koi_steff_err2,koi_slogg,koi_slogg_err1,koi_slogg_err2,koi_srad,koi_srad_err1,koi_srad_err2,ra,dec,koi_kepmag
5964,1,0,0,0,252.044440,0.027490,-0.027490,265.2010,0.0494,-0.0494,...,-136.0,4.621,0.041,-0.035,0.664,0.057,-0.059,292.79022,41.948639,15.884
9410,1,0,0,0,371.518520,0.015790,-0.015790,317.6836,0.0339,-0.0339,...,-206.0,4.377,0.101,-0.203,1.089,0.364,-0.145,293.06400,45.034210,13.731
4204,0,0,1,0,8.038670,0.000114,-0.000114,135.3098,0.0123,-0.0123,...,-181.0,4.485,0.050,-0.200,0.975,0.282,-0.101,290.51785,41.238762,14.999
5933,0,0,0,0,18.782160,0.000406,-0.000406,147.8508,0.0148,-0.0148,...,-167.0,4.488,0.048,-0.290,0.940,0.386,-0.087,291.76413,41.860130,14.043
6996,1,0,1,0,12.429716,0.000472,-0.000472,141.2846,0.0420,-0.0420,...,-200.0,4.534,0.037,-0.213,0.905,0.281,-0.088,297.52072,40.585419,15.842


# Pre-processing

Scale the data using LabelEncoder and MinMaxScaler

In [6]:
from sklearn.preprocessing import LabelEncoder, MinMaxScaler
X_scaler = MinMaxScaler().fit(X_train)

X_train_scaled = X_scaler.transform(X_train)
X_test_scaled = X_scaler.transform(X_test)

In [ ]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.layers import Dense

In [8]:
label_encoder = LabelEncoder()
label_encoder.fit(y_train)
encoded_y_train = label_encoder.transform(y_train)
encoded_y_test = label_encoder.transform(y_test)

y_train_categorical = to_categorical(encoded_y_train)
y_test_categorical = to_categorical(encoded_y_test)


In [9]:
y_train_categorical.shape

(6558, 3)

# Create and train the Deep Learning Model using Keras

In [10]:
model = Sequential()
model.add(Dense(units=100, activation='relu', input_dim=40))
model.add(Dense(units=100, activation='relu'))
model.add(Dense(units=3, activation='softmax'))

In [11]:
# Compile and fit the model
model.compile(optimizer='adam',
              loss='categorical_crossentropy',
              metrics=['accuracy'])

In [12]:
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense (Dense)                (None, 100)               4100      
_________________________________________________________________
dense_1 (Dense)              (None, 100)               10100     
_________________________________________________________________
dense_2 (Dense)              (None, 3)                 303       
Total params: 14,503
Trainable params: 14,503
Non-trainable params: 0
_________________________________________________________________


In [13]:
model.fit(
    X_train_scaled,
    y_train_categorical,
    epochs=60,
    shuffle=True,
    verbose=2
)

Epoch 1/60
 - 2s - loss: 0.5075 - acc: 0.7463
Epoch 2/60
 - 1s - loss: 0.3673 - acc: 0.8132
Epoch 3/60
 - 1s - loss: 0.3501 - acc: 0.8216
Epoch 4/60
 - 1s - loss: 0.3432 - acc: 0.8245
Epoch 5/60
 - 1s - loss: 0.3333 - acc: 0.8365
Epoch 6/60
 - 1s - loss: 0.3332 - acc: 0.8346
Epoch 7/60
 - 1s - loss: 0.3186 - acc: 0.8507
Epoch 8/60
 - 1s - loss: 0.3141 - acc: 0.8545
Epoch 9/60
 - 1s - loss: 0.3079 - acc: 0.8605
Epoch 10/60
 - 1s - loss: 0.3074 - acc: 0.8637
Epoch 11/60
 - 1s - loss: 0.3014 - acc: 0.8646
Epoch 12/60
 - 1s - loss: 0.3062 - acc: 0.8599
Epoch 13/60
 - 1s - loss: 0.2931 - acc: 0.8695
Epoch 14/60
 - 1s - loss: 0.2929 - acc: 0.8660
Epoch 15/60
 - 1s - loss: 0.2868 - acc: 0.8733
Epoch 16/60
 - 1s - loss: 0.2846 - acc: 0.8745
Epoch 17/60
 - 1s - loss: 0.2877 - acc: 0.8702
Epoch 18/60
 - 1s - loss: 0.2781 - acc: 0.8789
Epoch 19/60
 - 1s - loss: 0.2797 - acc: 0.8771
Epoch 20/60
 - 1s - loss: 0.2854 - acc: 0.8713
Epoch 21/60
 - 1s - loss: 0.2757 - acc: 0.8786
Epoch 22/60
 - 1s - lo

In [14]:
model_loss, model_accuracy = model.evaluate(
    X_test_scaled, y_test_categorical, verbose=2)
print(
    f"Normal Neural Network - Loss: {model_loss}, Accuracy: {model_accuracy}")

Normal Neural Network - Loss: 0.3007972549650501, Accuracy: 0.8732845378598268


In [ ]:
encoded_predictions = model.predict_classes(X_test_scaled[:5])
prediction_labels = label_encoder.inverse_transform(encoded_predictions)

In [16]:
print(f"Predicted classes: {prediction_labels}")
print(f"Actual Labels: {list(y_test[:5])}")

Predicted classes: ['False_Positive' 'CONFIRMED' 'CONFIRMED' 'CANDIDATE' 'CONFIRMED']
Actual Labels: ['False_Positive', 'CANDIDATE', 'CONFIRMED', 'CANDIDATE', 'CANDIDATE']
